In [3]:
import pandas as pd

In [2]:
melon = pd.read_csv("../Data/melon_top2000.csv")

In [ ]:
melon

In [4]:
# !pip install selenium

In [8]:
# !pip install webdriver-manager

  Using cached python_dotenv-0.20.0-py3-none-any.whl (17 kB)


In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import urllib.request as request
import time

In [5]:
chrome_options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

# 1, 451, 50
# for문으로 인덱스 돌리기

total_lists = []
for page in list(range(1, 2000, 50)):
  
  driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
  driver.get('https://www.melon.com/genre/song_list.htm#params%5BgnrCode%5D=GN0100&params%5BdtlGnrCode%5D=&params%5BorderBy%5D=POP&params%5BsteadyYn%5D=N&po=pageObj&startIndex='+ str(page))

  page_source = driver.page_source
  soup = BeautifulSoup(page_source, "html.parser")
  
  songIds = soup.select("#frm > div > table > tbody td:nth-child(4) > div > a")
  total_lists = total_lists + songIds

[WDM] - Downloading: 100%|██████████| 6.13M/6.13M [00:00<00:00, 9.13MB/s]


In [6]:
# songId 가져오기
songId_list = []
for songId in total_lists:
    songId_list.append(songId['href'].split('\'')[1])

len(songId_list)

2000

In [7]:
# 1000곡씩 나누기
songId_list_1000 =  songId_list[:1000]
songId_list_2000 =  songId_list[1000:]

In [8]:
len(songId_list[1000:])

1000

In [9]:
# 가수명, 노래제목, 발매일, 가사, 좋아요 수 리스트 변수 생성

singer_list = []
singName_list = []
date_list = []
lyrics_list = []
like_list = []
songImage_list =[]

# for문으로 가수명, 노래제목, 발매일, 가사, 좋아요 수 리스트 변수에 내용 담고 저장하기

chrome_options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

for songId in songId_list_2000:
    
    driver.get(f'https://www.melon.com/song/detail.htm?songId={songId}')

    page_source = driver.page_source
    soup = BeautifulSoup(page_source, "html.parser")
    try:
        time.sleep(2)
        # 가수명 가져오기
        artist_name = soup.select_one('#downloadfrm > div > div > div.entry > div.info > div.artist > a > span:nth-child(1) ')
        singer_list.append(artist_name.string)

        # 노래제목 가져오기
        song_name = soup.select_one("#downloadfrm > div > div > div.entry > div.info > div.song_name ")
        singName_list.append(song_name.text.strip().split('\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t')[1])

        # 발매일 가져오기
        date = soup.select_one('#downloadfrm > div > div > div.entry > div.meta > dl > dd:nth-child(4)')
        date_list.append(date.string)

        # 가사 가져오기
        lyric1 = str(soup.select_one('#d_video_summary')).replace('<br/>',' ')\
                                                         .replace('<div class="lyric" id="d_video_summary"><!-- height:auto; 로 변경시, 확장됨 -->',' ')\
                                                         .replace('</div>',' ').replace('\t','').replace('\n',' ')
        lyric = lyric1.replace('  ', ' ')
        lyrics_list.append(lyric)

        # 좋아요수 가져오기
        like = soup.select_one("#d_like_count")
        like_list.append(like.text)

        # 이미지 링크 가져오기
        # 앨범 사진 가져오기
        song_image = soup.select_one('#downloadfrm > div > div > div.thumb > a.image_typeAll > img')["src"]
        songImage_list.append(song_image)
        time.sleep(2)
    except:
        singer_list.append(0)
        singName_list.append(0)
        date_list.append(0)
        lyrics_list.append(0)
        like_list.append(0)
        songImage_list.append(0)
        time.sleep(1)

In [12]:
print(len(singer_list))
print(len(singName_list))
print(len(date_list))
print(len(lyrics_list))
print(len(like_list))
print(len(songImage_list))
print(len(songId_list_2000))

1000
1000
1000
1000
1000
1000
1000


In [13]:
ballad = {'가수명':singer_list, '노래제목':singName_list, '발매일':date_list, '가사':lyrics_list, '좋아요수':like_list, '앨범사진':songImage_list, 'songId':songId_list_2000}

In [14]:
import pandas as pd

In [15]:
ballad_1001_2000 =  pd.DataFrame(ballad)

In [16]:
ballad_1001_2000.tail()

,가수명,노래제목,발매일,가사,좋아요수,앨범사진,songId
995,터보,회상 (December),1997,겨울 오면은 우리 둘이서 항상 왔었던 바닷가 시린 바람과 하얀 파도는 예전 그대로...,"12,742",https://cdnimg.melon.co.kr/cm/album/images/003...,1606413
996,이석훈,고백,2011.06.02,좋은 사람 만나 행복해야 해 너는 웃고 있지만 나는 웃을 수가 없는걸 그래 난 알...,"18,569",https://cdnimg.melon.co.kr/cm/album/images/012...,3426230
997,다비치,나의 첫사랑,2022.05.16,그때의 나는 너를 만나서 하나부터 열까지 모든 순간들이 소중했어 내 첫사랑 잊고 ...,925,https://cdnimg.melon.co.kr/cm2/album/images/10...,35145139
998,미스에스,이 나이 먹고 뭐했길래 (Feat. 정슬기),2010.09.10,다시 처음으로 돌아갈 수 있다면 아마 너를 사랑하는 일은 안했을텐데 그 날밤 그 ...,"10,416",https://cdnimg.melon.co.kr/cm/album/images/010...,2933662
999,하은 (포맨),혼코노 (Honkono),2019.10.17,이 동전 하나로 뭘 할 수 있을까 고민 고민하다가 나 혼자 이곳에 왔어 익숙한 번...,"15,260",https://cdnimg.melon.co.kr/cm2/album/images/10...,32104350


In [17]:
len(ballad_1001_2000['노래제목'].unique())

964

In [18]:
ballad_1001_2000.isnull().sum()

가수명       0
노래제목      0
발매일       0
가사        0
좋아요수      0
앨범사진      0
songId    0
dtype: int64

In [19]:
ballad_1001_2000['좋아요수'] = ballad_1001_2000['좋아요수'].str.replace(',','')

In [20]:
ballad_1001_2000['발매일'] = ballad_1001_2000['발매일'].str.replace('.', '-')

C:\Users\user\AppData\Local\Temp/ipykernel_25384/2744582719.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  ballad_1001_2000['발매일'] = ballad_1001_2000['발매일'].str.replace('.', '-')


In [21]:
ballad_1001_2000['발매일'] = pd.to_datetime(ballad_1001_2000['발매일'])

In [23]:
ballad_1001_2000['순위'] = ballad_1001_2000.index + 1001

In [25]:
ballad_1001_2000.tail()

,가수명,노래제목,발매일,가사,좋아요수,앨범사진,songId,순위
995,터보,회상 (December),1997-01-01,겨울 오면은 우리 둘이서 항상 왔었던 바닷가 시린 바람과 하얀 파도는 예전 그대로...,12742,https://cdnimg.melon.co.kr/cm/album/images/003...,1606413,1996
996,이석훈,고백,2011-06-02,좋은 사람 만나 행복해야 해 너는 웃고 있지만 나는 웃을 수가 없는걸 그래 난 알...,18569,https://cdnimg.melon.co.kr/cm/album/images/012...,3426230,1997
997,다비치,나의 첫사랑,2022-05-16,그때의 나는 너를 만나서 하나부터 열까지 모든 순간들이 소중했어 내 첫사랑 잊고 ...,925,https://cdnimg.melon.co.kr/cm2/album/images/10...,35145139,1998
998,미스에스,이 나이 먹고 뭐했길래 (Feat. 정슬기),2010-09-10,다시 처음으로 돌아갈 수 있다면 아마 너를 사랑하는 일은 안했을텐데 그 날밤 그 ...,10416,https://cdnimg.melon.co.kr/cm/album/images/010...,2933662,1999
999,하은 (포맨),혼코노 (Honkono),2019-10-17,이 동전 하나로 뭘 할 수 있을까 고민 고민하다가 나 혼자 이곳에 왔어 익숙한 번...,15260,https://cdnimg.melon.co.kr/cm2/album/images/10...,32104350,2000


In [26]:
# ballad_1001_2000.to_csv("../Data/mel_bal_1001_2000.csv")